# 基本概念

张量	|角色	|作用
---|---|---
segment_ids	|输入标记，区分问题与上下文	|告诉模型答案只能从 segment_ids=1 部分选取
start_positions	|答案起点标签，模型目标	|模型预测答案的 第一个词 位置
end_positions	|答案终点标签，模型目标	|模型预测答案的 最后一个词 位置

在 BERT 问答系统 中，start_positions 和 end_positions 作为 目标标签 是 传入模型的训练过程 的，而不是作为输入序列的一部分传递给模型。模型会基于输入的文本预测这些位置，最终根据预测值与真实标签（即 start_positions 和 end_positions）计算损失（loss）来进行优化。

详细解释：  
- input_ids 和 segment_ids 是模型的 输入数据，它们描述了问题和上下文的结构。
- start_positions 和 end_positions 作为 标签 (target labels) 是 用于监督学习的目标值。它们告诉模型 正确的答案起始位置 和 结束位置。
  
训练过程中的步骤：  

输入数据传递给模型：  
- 在训练过程中，模型接收输入数据（input_ids 和 segment_ids）并进行前向传播，输出预测的 start_logits 和 end_logits。

模型的输出：

- start_logits 是模型对每个位置为答案起点的预测概率（一个概率分布）。
- end_logits 是模型对每个位置为答案终点的预测概率（一个概率分布）。
损失计算：  
- 模型的输出（start_logits 和 end_logits）将与 真实的答案起止位置（start_positions 和 end_positions）进行比较，计算损失。
- 损失函数通常使用 CrossEntropyLoss，计算模型输出和真实标签之间的误差。

反向传播和优化：  
- 通过损失计算，模型将更新其参数，以提高答案预测的准确性。



# 代码实现

In [20]:
import pandas as pd

In [21]:
data = pd.read_csv("../../../../../data/03.nlp/ChineseNlpCorpus/datasets/baoxianzhidao/baoxianzhidao_filter.csv")

In [22]:
data

,title,question,reply,is_best
0,最近在安邦长青树中看到什么豁免，这个是什么意思？,NaN,您好，这个是重疾险中给予投保者的一项权利，安*长青树保障责任规定，投保者可以享受多次赔付，豁...,1
1,和老婆利用假期去澳*探亲，但是第一次去不大熟悉，有没有相关保险呢？,NaN,您好，HUTS保险中的乐游全球（探亲版）-慧择旅游保险澳新计划是澳*新西兰探亲专属保障，承保...,0
2,HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了,NaN,您好，水上运动比赛，尤其是带有奖金的比赛一般承保的公司比较少。不过，HUTS保险中的众行天下...,1
3,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,NaN,您好，端午出行的人比较多，而且自驾存在一定风险，所以有保险意识还是很好的。考虑到价格以及保障...,1
4,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,NaN,不到10块钱………………,0
...,...,...,...,...
8357,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,你好！每年的保费不要超过年收入的20%,0
8358,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,可以退保费的意外险下载注册平安app，填邀请码自已投保里面有N个一百万身价,0
8359,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,你好平安守护星是一款分红型产品也可以做为教育金为主是一款少儿产品如有意向可以私聊我具体了解,0
8360,23岁买什么保险好啊？,我今年刚刚23岁，大学毕业刚开始工作，想给自己买份保险，不知道有什么保险好啊？,根据您提供的信息，建议您购买一份综合意外保险。保障普通意外、意外医疗、交通意外、住院津贴等等...,1


In [23]:
data = data[data['is_best']==1]

In [24]:
# 用 'title' 列填充 'question' 列中的空值
data.loc[data['question'].isna(), 'question'] = data['title']

In [25]:
data

,title,question,reply,is_best
0,最近在安邦长青树中看到什么豁免，这个是什么意思？,最近在安邦长青树中看到什么豁免，这个是什么意思？,您好，这个是重疾险中给予投保者的一项权利，安*长青树保障责任规定，投保者可以享受多次赔付，豁...,1
2,HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了,HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了,您好，水上运动比赛，尤其是带有奖金的比赛一般承保的公司比较少。不过，HUTS保险中的众行天下...,1
3,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,您好，端午出行的人比较多，而且自驾存在一定风险，所以有保险意识还是很好的。考虑到价格以及保障...,1
5,端午我们准备要举行赛龙舟，说是要份保险，什么好,端午我们准备要举行赛龙舟，说是要份保险，什么好,您好，赛龙舟是一项比较传统的活动，很有意义。不过由于是在水上活动，建议安全保障还要做足，HU...,1
6,老婆买了安*长*树，她在网上投保的，以后缴费怎么办,老婆买了安*长*树，她在网上投保的，以后缴费怎么办,您好，这点是不用担心的。投保后保险公司会在约定的保险费交纳日从消费者购买时填写的银行账号中划...,1
...,...,...,...,...
8345,"约了几个朋友去青*高*玩,不过怕遇到意外,有什么保险建议吗?","约了几个朋友去青*高*玩,不过怕遇到意外,有什么保险建议吗?",您好，青*高*的确由于特殊的地理环境，对于旅游者而言是有相当大的挑战的。所以出门前做好必要的...,1
8348,为什么我老公去买重疾险说是六类职业不给买？康惠保也是这样的吗,为什么我老公去买重疾险说是六类职业不给买？康惠保也是这样的吗,您好，重疾险之所以对职业有区分，是以为个人的职业对投保者身体健康影响比较大，一般情况下，市面...,1
8353,出国意外险有多少种？,出国意外险有多少种？一次最多可以买多少分？,出国意外险也就是境外保险，一般按照出国目的的不同，可以分为来说境外旅游保险、出国留学保险、出...,1
8354,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,您的保险意识非常的好，您和您丈夫目前的确最迫切的保险是意外险和重疾险。根据你们的收入情况，推...,1


In [26]:
data = data.drop(['title', 'is_best'], axis=1).reset_index(drop=True)

In [27]:
data

,question,reply
0,最近在安邦长青树中看到什么豁免，这个是什么意思？,您好，这个是重疾险中给予投保者的一项权利，安*长青树保障责任规定，投保者可以享受多次赔付，豁...
1,HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了,您好，水上运动比赛，尤其是带有奖金的比赛一般承保的公司比较少。不过，HUTS保险中的众行天下...
2,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,您好，端午出行的人比较多，而且自驾存在一定风险，所以有保险意识还是很好的。考虑到价格以及保障...
3,端午我们准备要举行赛龙舟，说是要份保险，什么好,您好，赛龙舟是一项比较传统的活动，很有意义。不过由于是在水上活动，建议安全保障还要做足，HU...
4,老婆买了安*长*树，她在网上投保的，以后缴费怎么办,您好，这点是不用担心的。投保后保险公司会在约定的保险费交纳日从消费者购买时填写的银行账号中划...
...,...,...
3783,"约了几个朋友去青*高*玩,不过怕遇到意外,有什么保险建议吗?",您好，青*高*的确由于特殊的地理环境，对于旅游者而言是有相当大的挑战的。所以出门前做好必要的...
3784,为什么我老公去买重疾险说是六类职业不给买？康惠保也是这样的吗,您好，重疾险之所以对职业有区分，是以为个人的职业对投保者身体健康影响比较大，一般情况下，市面...
3785,出国意外险有多少种？一次最多可以买多少分？,出国意外险也就是境外保险，一般按照出国目的的不同，可以分为来说境外旅游保险、出国留学保险、出...
3786,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,您的保险意识非常的好，您和您丈夫目前的确最迫切的保险是意外险和重疾险。根据你们的收入情况，推...


In [34]:
data.to_csv('../data/FAQ.csv',index=False)

In [35]:
pd.read_csv('../data/FAQ.csv')

,question,reply
0,最近在安邦长青树中看到什么豁免，这个是什么意思？,您好，这个是重疾险中给予投保者的一项权利，安*长青树保障责任规定，投保者可以享受多次赔付，豁...
1,HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了,您好，水上运动比赛，尤其是带有奖金的比赛一般承保的公司比较少。不过，HUTS保险中的众行天下...
2,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,您好，端午出行的人比较多，而且自驾存在一定风险，所以有保险意识还是很好的。考虑到价格以及保障...
3,端午我们准备要举行赛龙舟，说是要份保险，什么好,您好，赛龙舟是一项比较传统的活动，很有意义。不过由于是在水上活动，建议安全保障还要做足，HU...
4,老婆买了安*长*树，她在网上投保的，以后缴费怎么办,您好，这点是不用担心的。投保后保险公司会在约定的保险费交纳日从消费者购买时填写的银行账号中划...
...,...,...
3783,"约了几个朋友去青*高*玩,不过怕遇到意外,有什么保险建议吗?",您好，青*高*的确由于特殊的地理环境，对于旅游者而言是有相当大的挑战的。所以出门前做好必要的...
3784,为什么我老公去买重疾险说是六类职业不给买？康惠保也是这样的吗,您好，重疾险之所以对职业有区分，是以为个人的职业对投保者身体健康影响比较大，一般情况下，市面...
3785,出国意外险有多少种？一次最多可以买多少分？,出国意外险也就是境外保险，一般按照出国目的的不同，可以分为来说境外旅游保险、出国留学保险、出...
3786,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,您的保险意识非常的好，您和您丈夫目前的确最迫切的保险是意外险和重疾险。根据你们的收入情况，推...


# 23123

In [36]:
# 示例输入
question = "什么是保险？"
context = "保险是一种风险管理手段，指通过合同规定风险损失的补偿。"
answer = "风险管理手段"

In [39]:
from transformers import BertTokenizerFast

# 加载 BERT 中文分词器 (Fast 版)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# 示例输入
question = "什么是保险？"
context = "保险是一种风险管理手段，指通过合同规定风险损失的补偿。"
answer = "风险管理手段"

# 编码 question 和 context，返回偏移映射
inputs = tokenizer(
    question, 
    context, 
    max_length=128, 
    padding="max_length", 
    truncation=True, 
    return_offsets_mapping=True,  # 获取偏移映射
    return_tensors="pt"
)

# 提取偏移映射
offset_mapping = inputs["offset_mapping"][0].tolist()

# 获取答案在原始文本中的位置
start_char = context.find(answer)
end_char = start_char + len(answer)

# 初始化起止位置
start_position = end_position = None

# 遍历偏移映射，查找答案的位置
for idx, (start, end) in enumerate(offset_mapping):
    if start == start_char:
        start_position = idx
    if end == end_char:
        end_position = idx
        break

# 检查结果
print(f"Start Position: {start_position}, End Position: {end_position}")

Start Position: 13, End Position: 18


In [40]:
start_char

5

In [41]:
end_char

11

In [44]:
inputs["offset_mapping"]

tensor([[[ 0,  0],
         [ 0,  1],
         [ 1,  2],
         [ 2,  3],
         [ 3,  4],
         [ 4,  5],
         [ 5,  6],
         [ 0,  0],
         [ 0,  1],
         [ 1,  2],
         [ 2,  3],
         [ 3,  4],
         [ 4,  5],
         [ 5,  6],
         [ 6,  7],
         [ 7,  8],
         [ 8,  9],
         [ 9, 10],
         [10, 11],
         [11, 12],
         [12, 13],
         [13, 14],
         [14, 15],
         [15, 16],
         [16, 17],
         [17, 18],
         [18, 19],
         [19, 20],
         [20, 21],
         [21, 22],
         [22, 23],
         [23, 24],
         [24, 25],
         [25, 26],
         [26, 27],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0

In [45]:
print(inputs["input_ids"]) 

tensor([[ 101,  784,  720, 3221,  924, 7372, 8043,  102,  924, 7372, 3221,  671,
         4905, 7599, 7372, 5052, 4415, 2797, 3667, 8024, 2900, 6858, 6814, 1394,
         1398, 6226, 2137, 7599, 7372, 2938, 1927, 4638, 6133,  985,  511,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])


In [46]:
from transformers import BertTokenizerFast

# 加载分词器
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# 定义输入
question = "什么是保险？"
context = "保险是一种风险管理手段。"

# 编码输入
inputs = tokenizer(
    question, context, 
    max_length=128, 
    padding="max_length", 
    truncation=True, 
    return_token_type_ids=True,  # 获取 segment_ids
    return_tensors="pt"
)

# 查看结果
print("Input IDs:", inputs["input_ids"])
print("Token Type IDs (segment_ids):", inputs["token_type_ids"])


D:\Anaconda\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Input IDs: tensor([[ 101,  784,  720, 3221,  924, 7372, 8043,  102,  924, 7372, 3221,  671,
         4905, 7599, 7372, 5052, 4415, 2797, 3667,  511,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])
Token Type IDs (segment_ids): tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         

# 2313123

In [50]:
from transformers import BertTokenizerFast

# 加载分词器
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# 示例数据
questions = ["什么是保险？", "如何买保险？"]
contexts = [
    "保险是一种风险管理手段，旨在减轻不确定性带来的经济损失。",
    "买保险是个人或公司购买保障计划以应对未来可能发生的风险。"
]
answers = ["风险管理手段", "购买保障计划"]

# 批量编码
inputs = tokenizer(
    questions, contexts, 
    max_length=64, 
    padding="max_length", 
    truncation=True, 
    return_offsets_mapping=True, 
    return_tensors="pt"
)

# 提取必要的信息
input_ids = inputs["input_ids"]
token_type_ids = inputs["token_type_ids"]
offset_mappings = inputs["offset_mapping"]

start_positions, end_positions = [], []

for i, (context, answer) in enumerate(zip(contexts, answers)):
    answer_start = context.find(answer)
    answer_end = answer_start + len(answer)
    
    if answer_start == -1:
        print(f"答案未找到: {answer} 在 {context} 中")
        start_positions.append(-1)
        end_positions.append(-1)
        continue
    
    # 查找 token 索引
    found_start, found_end = False, False
    for idx, (start, end) in enumerate(offset_mappings[i].tolist()):
        if start == answer_start and not found_start:
            start_positions.append(idx)
            found_start = True
        if end == answer_end and found_start:
            end_positions.append(idx)
            found_end = True
            break

    if not found_start or not found_end:
        print(f"答案位置未找到: {answer}")

# 打印结果
print("Input IDs:", input_ids)
print("Segment IDs:", token_type_ids)
print("Start Positions:", start_positions)
print("End Positions:", end_positions)

Input IDs: tensor([[ 101,  784,  720, 3221,  924, 7372, 8043,  102,  924, 7372, 3221,  671,
         4905, 7599, 7372, 5052, 4415, 2797, 3667, 8024, 3192, 1762, 1121, 6768,
          679, 4802, 2137, 2595, 2372, 3341, 4638, 5307, 3845, 2938, 1927,  511,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [ 101, 1963,  862,  743,  924, 7372, 8043,  102,  743,  924, 7372, 3221,
          702,  782, 2772, 1062, 1385, 6579,  743,  924, 7397, 6369, 1153,  809,
         2418, 2190, 3313, 3341, 1377, 5543, 1355, 4495, 4638, 7599, 7372,  511,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
Segment IDs: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1

In [54]:
from transformers import AutoTokenizer

# 初始化分词器
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# 示例数据
questions = ["法国的首都是哪里？", "谁写了哈姆雷特？"]
contexts = ["法国的首都是巴黎。", "威廉·莎士比亚于1603年写了哈姆雷特。"]
answers = ["巴黎", "威廉·莎士比亚"]

# 存储结果
start_positions = []
end_positions = []

# 遍历数据
for question, context, answer in zip(questions, contexts, answers):
    # 编码 question 和 context
    inputs = tokenizer(
        question, 
        context, 
        max_length=128, 
        padding="max_length", 
        truncation=True, 
        return_offsets_mapping=True,  # 获取偏移映射
        return_tensors="pt"
    )

    # 提取偏移映射和输入ID
    offset_mapping = inputs["offset_mapping"][0].tolist()
    input_ids = inputs["input_ids"][0].tolist()

    # 获取答案的起止字符位置
    answer_start = context.find(answer)
    answer_end = answer_start + len(answer)

    # 标记是否找到答案
    found_start = found_end = False

    # 遍历偏移映射查找起止位置
    for idx, (start, end) in enumerate(offset_mapping):
        if not found_start and start == answer_start:
            start_positions.append(idx)
            found_start = True
        if found_start and not found_end and end == answer_end:
            end_positions.append(idx)
            found_end = True
            break

    # 检查未找到的情况
    if not found_start or not found_end:
        start_positions.append(-1)
        end_positions.append(-1)

# 打印结果
print(f"起始位置: {start_positions}")
print(f"结束位置: {end_positions}")

起始位置: [7, 0]
结束位置: [8, 7]


In [55]:
print("Input IDs:", input_ids)
print("Segment IDs:", token_type_ids)
print("Start Positions:", start_positions)
print("End Positions:", end_positions)

Input IDs: [101, 6443, 1091, 749, 1506, 1990, 7440, 4294, 8043, 102, 2014, 2442, 185, 5801, 1894, 3683, 762, 754, 8522, 8152, 2399, 1091, 749, 1506, 1990, 7440, 4294, 511, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Segment IDs: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Start Positions: [7, 0]
End Positions: [8, 7]


In [53]:
inputs["offset_mapping"]

tensor([[[ 0,  0],
         [ 0,  1],
         [ 1,  2],
         [ 2,  3],
         [ 3,  4],
         [ 4,  5],
         [ 5,  6],
         [ 6,  7],
         [ 7,  8],
         [ 0,  0],
         [ 0,  1],
         [ 1,  2],
         [ 2,  3],
         [ 3,  4],
         [ 4,  5],
         [ 5,  6],
         [ 6,  7],
         [ 7,  8],
         [ 8, 11],
         [11, 12],
         [12, 13],
         [13, 14],
         [14, 15],
         [15, 16],
         [16, 17],
         [17, 18],
         [18, 19],
         [19, 20],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0],
         [ 0

In [56]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class QADataSet(Dataset):
    def __init__(self, questions, contexts, answers, tokenizer_name="bert-base-chinese", max_length=128):
        self.questions = questions
        self.contexts = contexts
        self.answers = answers
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.contexts[idx]
        answer = self.answers[idx]

        # 编码输入
        inputs = self.tokenizer(
            question, 
            context, 
            max_length=self.max_length, 
            padding="max_length", 
            truncation=True, 
            return_offsets_mapping=True, 
            return_tensors="pt"
        )

        # 提取偏移映射和输入ID
        offset_mapping = inputs["offset_mapping"][0].tolist()
        input_ids = inputs["input_ids"][0].tolist()

        # 查找答案在上下文中的位置
        answer_start = context.find(answer)
        answer_end = answer_start + len(answer)

        # 初始化起止位置
        start_position = -1
        end_position = -1

        # 遍历偏移映射，找到起止位置
        for idx, (start, end) in enumerate(offset_mapping):
            if start == answer_start and start_position == -1:
                start_position = idx
            if end == answer_end and start_position != -1:
                end_position = idx
                break
        
        # 返回样本
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "token_type_ids": inputs["token_type_ids"].squeeze(),
            "start_positions": torch.tensor(start_position),
            "end_positions": torch.tensor(end_position)
        }


In [57]:
# 示例数据
questions = ["法国的首都是哪里？", "谁写了哈姆雷特？"]
contexts = ["法国的首都是巴黎。", "威廉·莎士比亚于1603年写了哈姆雷特。"]
answers = ["巴黎", "威廉·莎士比亚"]

# 加载数据集
dataset = QADataSet(questions, contexts, answers)

# 检查样本
for data in dataset:
    print(f"Input IDs: {data['input_ids']}")
    print(f"Attention Mask: {data['attention_mask']}")
    print(f"Token Type IDs: {data['token_type_ids']}")
    print(f"Start Position: {data['start_positions']}")
    print(f"End Position: {data['end_positions']}")
    print("-" * 50)

Input IDs: tensor([ 101, 3791, 1744, 4638, 7674, 6963, 3221, 1525, 7027, 8043,  102, 3791,
        1744, 4638, 7674, 6963, 3221, 2349, 7944,  511,  102,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# 数据处理

In [58]:
data = pd.read_csv("../../../../../data/03.nlp/ChineseNlpCorpus/datasets/baoxianzhidao/baoxianzhidao_filter.csv")

In [59]:
data

,title,question,reply,is_best
0,最近在安邦长青树中看到什么豁免，这个是什么意思？,NaN,您好，这个是重疾险中给予投保者的一项权利，安*长青树保障责任规定，投保者可以享受多次赔付，豁...,1
1,和老婆利用假期去澳*探亲，但是第一次去不大熟悉，有没有相关保险呢？,NaN,您好，HUTS保险中的乐游全球（探亲版）-慧择旅游保险澳新计划是澳*新西兰探亲专属保障，承保...,0
2,HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了,NaN,您好，水上运动比赛，尤其是带有奖金的比赛一般承保的公司比较少。不过，HUTS保险中的众行天下...,1
3,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,NaN,您好，端午出行的人比较多，而且自驾存在一定风险，所以有保险意识还是很好的。考虑到价格以及保障...,1
4,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,NaN,不到10块钱………………,0
...,...,...,...,...
8357,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,你好！每年的保费不要超过年收入的20%,0
8358,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,可以退保费的意外险下载注册平安app，填邀请码自已投保里面有N个一百万身价,0
8359,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,你好平安守护星是一款分红型产品也可以做为教育金为主是一款少儿产品如有意向可以私聊我具体了解,0
8360,23岁买什么保险好啊？,我今年刚刚23岁，大学毕业刚开始工作，想给自己买份保险，不知道有什么保险好啊？,根据您提供的信息，建议您购买一份综合意外保险。保障普通意外、意外医疗、交通意外、住院津贴等等...,1


In [60]:
data['title']

0                最近在安邦长青树中看到什么豁免，这个是什么意思？
1       和老婆利用假期去澳*探亲，但是第一次去不大熟悉，有没有相关保险呢？
2           HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了
3              计划端午节和男朋友自驾去九*山，买保险三天要多少钱？
4              计划端午节和男朋友自驾去九*山，买保险三天要多少钱？
                      ...                
8357                          如何为一家三口买保险？
8358                          如何为一家三口买保险？
8359                          如何为一家三口买保险？
8360                          23岁买什么保险好啊？
8361                          23岁买什么保险好啊？
Name: title, Length: 8362, dtype: object

In [62]:
# 按分类分组，将其他列合并为列表
result = data.groupby('title').agg(list).reset_index()
print(result)

                      title  \
0          **哈*的失业金可以在南*领取吗   
1         *光保险在贵*买的车险单掉了怎样补   
2         0-5岁的孩子现在买什么样的保险好   
3           021###11是平安保险吗？   
4          0免赔额的百万医疗险值不值得买？   
...                     ...   
4109  高速追尾对方全责报案后车修好了能索要赔偿吗   
4110              鸿运安行保险怎么样   
4111    黑*江*口*北牌照买车险需要什么手续?   
4112          齐*牌照在哈*滨*车*行*   
4113        ，保险是真的吗？寿险包括那些？   

                                               question  \
0                                       [nan, nan, nan]   
1                                   [*光保险在贵*买的车险单掉了怎样补]   
2                                            [nan, nan]   
3     [告诉他姓名和身份证了。没说银行卡，会不会从银行调我的卡号收费, 告诉他姓名和身份证了。没说...   
4                                            [nan, nan]   
...                                                 ...   
4109                                         [nan, nan]   
4110                                              [nan]   
4111                                    [nan, nan, nan]   
4112                                 

In [63]:
pd.DataFrame(result)

,title,question,reply,is_best
0,**哈*的失业金可以在南*领取吗,"[nan, nan, nan]",[可以领取，详细原因如下;可以异地领取失业金流程：失业人员户籍所在地与参保地不一致的，按照“...,"[1, 0, 0]"
1,*光保险在贵*买的车险单掉了怎样补,[*光保险在贵*买的车险单掉了怎样补],[车险保单是购买车险的凭证，保单丢失可以去保险公司进行补办。车主在补办保单时，只需要带上本人...,[1]
2,0-5岁的孩子现在买什么样的保险好,"[nan, nan]",[主要投保是三类：意外险、健康险和教育金保险。其中意外险对于会走会跑的孩子来说最为重要，而健...,"[1, 0]"
3,021###11是平安保险吗？,"[告诉他姓名和身份证了。没说银行卡，会不会从银行调我的卡号收费, 告诉他姓名和身份证了。没说...",[您好：很高兴为您提供咨询服务是平安的上*营*部电话，可能是送您意外险，才问了您的姓名和身份...,"[1, 0]"
4,0免赔额的百万医疗险值不值得买？,"[nan, nan]",[在保险精算师的眼里，一万元的免赔额远远要比100万元的保额更值钱。所以免赔额越低，就意味着...,"[1, 0]"
...,...,...,...,...
4109,高速追尾对方全责报案后车修好了能索要赔偿吗,"[nan, nan]",[可以索赔。不过最好是在定损以后修理进行索赔。一般在汽车被追尾后，责任由交警出具的交通事故责...,"[1, 0]"
4110,鸿运安行保险怎么样,[nan],[平安鸿运安行保险是一个电销寿险，产品是根据人群来判断是否合适的，个人建议买保险就是要买终身...,[1]
4111,黑*江*口*北牌照买车险需要什么手续?,"[nan, nan, nan]",[车辆办理保险的时候，需要车主向保险公司提供相应的文件证明。车主需要提供车辆出厂合格证，购车...,"[0, 0, 0]"
4112,齐*牌照在哈*滨*车*行*,[齐*牌照在哈*滨*车*行*],[在理论上是可以的，现在我国的保险公司为了加强自己公司的竞争力度，在国家的各个城市都开设分公...,[1]


In [70]:
# 按分类分组，仅合并描述列为列表
result = data.groupby('title').agg({'reply': list}).reset_index()
print(result)

                      title                                              reply
0          **哈*的失业金可以在南*领取吗  [可以领取，详细原因如下;可以异地领取失业金流程：失业人员户籍所在地与参保地不一致的，按照“...
1         *光保险在贵*买的车险单掉了怎样补  [车险保单是购买车险的凭证，保单丢失可以去保险公司进行补办。车主在补办保单时，只需要带上本人...
2         0-5岁的孩子现在买什么样的保险好  [主要投保是三类：意外险、健康险和教育金保险。其中意外险对于会走会跑的孩子来说最为重要，而健...
3           021###11是平安保险吗？  [您好：很高兴为您提供咨询服务是平安的上*营*部电话，可能是送您意外险，才问了您的姓名和身份...
4          0免赔额的百万医疗险值不值得买？  [在保险精算师的眼里，一万元的免赔额远远要比100万元的保额更值钱。所以免赔额越低，就意味着...
...                     ...                                                ...
4109  高速追尾对方全责报案后车修好了能索要赔偿吗  [可以索赔。不过最好是在定损以后修理进行索赔。一般在汽车被追尾后，责任由交警出具的交通事故责...
4110              鸿运安行保险怎么样  [平安鸿运安行保险是一个电销寿险，产品是根据人群来判断是否合适的，个人建议买保险就是要买终身...
4111    黑*江*口*北牌照买车险需要什么手续?  [车辆办理保险的时候，需要车主向保险公司提供相应的文件证明。车主需要提供车辆出厂合格证，购车...
4112          齐*牌照在哈*滨*车*行*  [在理论上是可以的，现在我国的保险公司为了加强自己公司的竞争力度，在国家的各个城市都开设分公...
4113        ，保险是真的吗？寿险包括那些？  [您好！在正规保险公司买的都是真的。根据保障的不同可以分为商业养老保险，少儿保险，理财保险，...

[4114 rows x 2 columns]


In [71]:
pd.DataFrame(result)

,title,reply
0,**哈*的失业金可以在南*领取吗,[可以领取，详细原因如下;可以异地领取失业金流程：失业人员户籍所在地与参保地不一致的，按照“...
1,*光保险在贵*买的车险单掉了怎样补,[车险保单是购买车险的凭证，保单丢失可以去保险公司进行补办。车主在补办保单时，只需要带上本人...
2,0-5岁的孩子现在买什么样的保险好,[主要投保是三类：意外险、健康险和教育金保险。其中意外险对于会走会跑的孩子来说最为重要，而健...
3,021###11是平安保险吗？,[您好：很高兴为您提供咨询服务是平安的上*营*部电话，可能是送您意外险，才问了您的姓名和身份...
4,0免赔额的百万医疗险值不值得买？,[在保险精算师的眼里，一万元的免赔额远远要比100万元的保额更值钱。所以免赔额越低，就意味着...
...,...,...
4109,高速追尾对方全责报案后车修好了能索要赔偿吗,[可以索赔。不过最好是在定损以后修理进行索赔。一般在汽车被追尾后，责任由交警出具的交通事故责...
4110,鸿运安行保险怎么样,[平安鸿运安行保险是一个电销寿险，产品是根据人群来判断是否合适的，个人建议买保险就是要买终身...
4111,黑*江*口*北牌照买车险需要什么手续?,[车辆办理保险的时候，需要车主向保险公司提供相应的文件证明。车主需要提供车辆出厂合格证，购车...
4112,齐*牌照在哈*滨*车*行*,[在理论上是可以的，现在我国的保险公司为了加强自己公司的竞争力度，在国家的各个城市都开设分公...


In [72]:
pd.DataFrame(result)['reply'][0]

['可以领取，详细原因如下;可以异地领取失业金流程：失业人员户籍所在地与参保地不一致的，按照“自愿的原则”，可选将失业保险关系转入户籍所在地，然后在户口所在地领取失业金。个人转移失业保险关系分为两种类别：在职职工转移失业保险关系。这类情况需要在参保地失业保险经办机构开具失业保险缴费记录，具体内容包括：失业保险参保缴费的单位及起止时间，以及未在当地享受失业保险待遇的证明。凭此证可以作为失业保险视同缴费年限，待在户口所在地区缴纳失业保险费满一年并符合领取失业保险金条件时，将失业保险费年限与户口所在地的失业保险缴费年限合并计算。失业后的失业保险关系转移。这类情况需要参保地失业保险经办机构联系，将应享受的失业保险金和职业培训费、职业介绍费，以及应由失业保险基金缴纳的基本医疗保险费转移至户口所在地，由户口所在地发放失业保险金、缴纳基本医疗保险费。',
 '领取失业金在退工一个月的时间内把档案退回原籍做失业登记,回原籍领取,逾期不能领取。而且已经就业,失业金停发。想经更具体的了解，推荐在律师咨询贴吧，可以去详询一些律师们。',
 '失业金异地领取之前，要先到转出地经办机构提申请，符合异地享受失业保险待遇条件的失业人员，可以到用人单位所在地的失业保险经办机构办理失业保险关系转迁。具体程序为：失业人员到转出地失业保险经办机构申请异地享受失业保险待遇，转出地失业保险经办机构出具转迁证明及其他相关材料，告知失业人员后，交转入地失业保险经办机构；转入地失业保险经办机构收到转迁证明后，向转出地失业保险经办机构出具接收证明；转出地失业保险经办机构收到接收证明后，按规定划转失业保险费用；转入地失业保险经办机构收到划转的相关资金后及时审核，按规定发放失业保险待遇。转出地、转入地失业保险经办机构每个工作环节15天。']

# shujuchuli 

In [73]:
data = pd.read_csv("../../../../../data/03.nlp/ChineseNlpCorpus/datasets/baoxianzhidao/baoxianzhidao_filter.csv")

In [74]:
data.loc[data['question'].isna(), 'question'] = data['title']

In [75]:
data

,title,question,reply,is_best
0,最近在安邦长青树中看到什么豁免，这个是什么意思？,最近在安邦长青树中看到什么豁免，这个是什么意思？,您好，这个是重疾险中给予投保者的一项权利，安*长青树保障责任规定，投保者可以享受多次赔付，豁...,1
1,和老婆利用假期去澳*探亲，但是第一次去不大熟悉，有没有相关保险呢？,和老婆利用假期去澳*探亲，但是第一次去不大熟悉，有没有相关保险呢？,您好，HUTS保险中的乐游全球（探亲版）-慧择旅游保险澳新计划是澳*新西兰探亲专属保障，承保...,0
2,HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了,HUTS中有没有适合帆船比赛的保险，我男朋友这周就要开始了,您好，水上运动比赛，尤其是带有奖金的比赛一般承保的公司比较少。不过，HUTS保险中的众行天下...,1
3,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,您好，端午出行的人比较多，而且自驾存在一定风险，所以有保险意识还是很好的。考虑到价格以及保障...,1
4,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,计划端午节和男朋友自驾去九*山，买保险三天要多少钱？,不到10块钱………………,0
...,...,...,...,...
8357,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,你好！每年的保费不要超过年收入的20%,0
8358,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,可以退保费的意外险下载注册平安app，填邀请码自已投保里面有N个一百万身价,0
8359,如何为一家三口买保险？,近段时间一直想给自己的小家买份保险，但是保险公司多，保险品种更多，看得眼花。所以想请各位专家...,你好平安守护星是一款分红型产品也可以做为教育金为主是一款少儿产品如有意向可以私聊我具体了解,0
8360,23岁买什么保险好啊？,我今年刚刚23岁，大学毕业刚开始工作，想给自己买份保险，不知道有什么保险好啊？,根据您提供的信息，建议您购买一份综合意外保险。保障普通意外、意外医疗、交通意外、住院津贴等等...,1


In [76]:
# 按分类分组，仅合并描述列为列表
result = data.groupby('question').agg({'reply': list}).reset_index()
print(result)

                               question  \
0                      **哈*的失业金可以在南*领取吗   
1                     *光保险在贵*买的车险单掉了怎样补   
2                     0-5岁的孩子现在买什么样的保险好   
3                      0免赔额的百万医疗险值不值得买？   
4     0免赔额的百万医疗险跟1万元免赔额的百万医疗险相比，究竟好在哪儿？   
...                                 ...   
4110                        麻烦说清楚一点，谢谢！   
4111                黑*江*口*北牌照买车险需要什么手续?   
4112                      齐*牌照在哈*滨*车*行*   
4113                    ，保险是真的吗？寿险包括那些？   
4114                                  ？   

                                                  reply  
0     [可以领取，详细原因如下;可以异地领取失业金流程：失业人员户籍所在地与参保地不一致的，按照“...  
1     [车险保单是购买车险的凭证，保单丢失可以去保险公司进行补办。车主在补办保单时，只需要带上本人...  
2     [主要投保是三类：意外险、健康险和教育金保险。其中意外险对于会走会跑的孩子来说最为重要，而健...  
3     [在保险精算师的眼里，一万元的免赔额远远要比100万元的保额更值钱。所以免赔额越低，就意味着...  
4     [其实，二者最大的区别就是免赔额度上，一个是0免赔，一个是有1万元的免赔额。0免赔与1万元免...  
...                                                 ...  
4110  [社会养老保险和商业养老保险的区别很大，各有各的好处，可以根据个人需求进行投保。关于商业养老...  
4111  [车辆办理保险的时候，需要

In [77]:
df = pd.DataFrame(result)
df

,question,reply
0,**哈*的失业金可以在南*领取吗,[可以领取，详细原因如下;可以异地领取失业金流程：失业人员户籍所在地与参保地不一致的，按照“...
1,*光保险在贵*买的车险单掉了怎样补,[车险保单是购买车险的凭证，保单丢失可以去保险公司进行补办。车主在补办保单时，只需要带上本人...
2,0-5岁的孩子现在买什么样的保险好,[主要投保是三类：意外险、健康险和教育金保险。其中意外险对于会走会跑的孩子来说最为重要，而健...
3,0免赔额的百万医疗险值不值得买？,[在保险精算师的眼里，一万元的免赔额远远要比100万元的保额更值钱。所以免赔额越低，就意味着...
4,0免赔额的百万医疗险跟1万元免赔额的百万医疗险相比，究竟好在哪儿？,[其实，二者最大的区别就是免赔额度上，一个是0免赔，一个是有1万元的免赔额。0免赔与1万元免...
...,...,...
4110,麻烦说清楚一点，谢谢！,[社会养老保险和商业养老保险的区别很大，各有各的好处，可以根据个人需求进行投保。关于商业养老...
4111,黑*江*口*北牌照买车险需要什么手续?,[车辆办理保险的时候，需要车主向保险公司提供相应的文件证明。车主需要提供车辆出厂合格证，购车...
4112,齐*牌照在哈*滨*车*行*,[在理论上是可以的，现在我国的保险公司为了加强自己公司的竞争力度，在国家的各个城市都开设分公...
4113,，保险是真的吗？寿险包括那些？,[您好！在正规保险公司买的都是真的。根据保障的不同可以分为商业养老保险，少儿保险，理财保险，...
